<a href="https://colab.research.google.com/github/hananbahtiti/Hybrid-Intrusion-detection-Systems/blob/main/NetworkPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.1 MB/s eta 0:00:00


In [ ]:
!cp /content/eve_tag.csv /content/drive/MyDrive/hybrid_IDS/dataset/

In [88]:
df = pd.read_csv('/content/eve.csv')
df.info()
df.head(3)
#df['alert'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286150 entries, 0 to 286149
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   proto           286148 non-null  object 
 1   event_type      286150 non-null  object 
 2   flow_id         286048 non-null  float64
 3   Date            286150 non-null  int64  
 4   hour            137399 non-null  float64
 5   minute          137399 non-null  float64
 6   second          137399 non-null  float64
 7   pkts_toclient   284361 non-null  float64
 8   bytes_toserver  284361 non-null  float64
 9   bytes_toclient  284361 non-null  float64
 10  tcp_flags       270253 non-null  float64
 11  alerted         279361 non-null  object 
 12  age             279361 non-null  float64
 13  reason          279361 non-null  object 
 14  pkts_toserver   284361 non-null  float64
 15  tcp_flags_ts    270253 non-null  float64
 16  state           279361 non-null  object 
dtypes: float64

,proto,event_type,flow_id,Date,hour,minute,second,pkts_toclient,bytes_toserver,bytes_toclient,tcp_flags,alerted,age,reason,pkts_toserver,tcp_flags_ts,state
0,TCP,alert,NaN,0,17.0,49.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TCP,alert,2.558362e+14,0,17.0,49.0,44.0,0.0,62.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,TCP,alert,2.558362e+14,0,17.0,49.0,44.0,0.0,62.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [77]:
df = pd.read_csv('/content/eve_tag_no_constant.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286150 entries, 0 to 286149
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   proto           286148 non-null  object 
 1   event_type      286150 non-null  object 
 2   flow_id         286048 non-null  float64
 3   Date            286150 non-null  int64  
 4   hour            137399 non-null  float64
 5   minute          137399 non-null  float64
 6   second          137399 non-null  float64
 7   pkts_toclient   284361 non-null  float64
 8   end             279361 non-null  object 
 9   bytes_toserver  284361 non-null  float64
 10  bytes_toclient  284361 non-null  float64
 11  tcp_flags       270253 non-null  float64
 12  alerted         279361 non-null  object 
 13  start           284361 non-null  object 
 14  age             279361 non-null  float64
 15  reason          279361 non-null  object 
 16  pkts_toserver   284361 non-null  float64
 17  tcp_flags_

,proto,event_type,flow_id,Date,hour,minute,second,pkts_toclient,end,bytes_toserver,bytes_toclient,tcp_flags,alerted,start,age,reason,pkts_toserver,tcp_flags_ts,state
0,TCP,alert,NaN,0,17.0,49.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TCP,alert,2.558362e+14,0,17.0,49.0,44.0,0.0,NaN,62.0,0.0,NaN,NaN,2016-03-10T17:49:44.321710+0200,NaN,NaN,1.0,NaN,NaN
2,TCP,alert,2.558362e+14,0,17.0,49.0,44.0,0.0,NaN,62.0,0.0,NaN,NaN,2016-03-10T17:49:44.321710+0200,NaN,NaN,1.0,NaN,NaN
3,UDP,tftp,5.620575e+13,0,17.0,49.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TCP,flow,8.582737e+14,0,17.0,48.0,59.0,0.0,2016-03-10T17:49:15.789656+0200,60.0,0.0,0.0,False,2016-03-10T17:49:15.789656+0200,0.0,timeout,1.0,0.0,new


In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import math
import ast
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [ ]:
try:
    from tqdm import tqdm
    TQDM_AVAILABLE = True
except ImportError:
    TQDM_AVAILABLE = False

In [ ]:
class Preprocessing:
  def __init__(self):
    pass

  def clean_and_save(self, file_path: str, output_path: str, threshold=80):
    try:
        df = pd.read_csv(file_path)
        total_rows = len(df)
        missing_ratios = (df.isnull().sum() / total_rows) * 100

        columns_to_drop = [col for col, ratio in missing_ratios.items() if ratio > threshold]

        print(f"Columns to drop: {columns_to_drop}")

        df = df.drop(columns=columns_to_drop, axis=1)
        df.to_csv(output_path, index=False)
        return f"Saved cleaned data to {output_path}"
    except Exception as e:
      return f"An error occurred: {e}"


  def columns_drop(self, file_path: str, columns_names: list, output_path: str):
    try:
        df = pd.read_csv(file_path)
        df = df.drop(columns=columns_names, axis=1)  # Drop columns once
        df.to_csv(output_path, index=False)
        return f"Saved cleaned data to {output_path}"
    except Exception as e:
        return f"An error occurred: {e}"


  def data_encoding(self, file_path, columns_names: list):
    encoder = LabelEncoder()
    tqdm.pandas()

    try:
        df = pd.read_csv(file_path)  # قراءة الملف مرة واحدة فقط

        def encoding_function(x, col_name):
            tqdm.write(f"Encoding {col_name}...", end='\r')
            return encoder.transform([x])[0] if pd.notna(x) else -1

        for col in columns_names:
            df.loc[:, col] = df.loc[:, col].fillna("-1").astype(str)

            encoder.fit(df.loc[:, col])

            df.loc[:, col] = df.loc[:, col].progress_apply(
                lambda x: encoding_function(x, col)
            )

        output_path = file_path.replace(".csv", "_encoded.csv")
        df.to_csv(output_path, index=False)
        return f"Done. Encoded file saved to: {output_path}"

    except Exception as e:
        return f"An error occurred: {e}"



  def drop_constant_columns(self, file_path, output_path=None):
    try:
        df = pd.read_csv(file_path)
        constant_columns = [col for col in df.columns if df[col].nunique(dropna=True) == 1]
        print(f"Columns with constant value: {constant_columns}")

        df = df.drop(columns=constant_columns)
        if output_path is None:
          output_path = file_path.replace('.csv', '_no_constant.csv')
        df.to_csv(output_path, index=False)

        return f"Saved cleaned file to {output_path}"

    except Exception as e:
      return f"An error occurred: {e}"



  def show_column_types(self, file_path):
    try:
      columns_names = []
      df = pd.read_csv(file_path)
      [columns_names.append(col)  for col in df.columns if df[col].dtype == 'object']

      return columns_names
    except Exception as e:
      return f"An error occurred: {e}"




  @staticmethod
  def expand_dict_columns( file_path, column_names: list, output_file_path: str = None):
      try:
          df = pd.read_csv(file_path)

          all_keys = set()

          # Step 1: Extract all keys
          def extract_keys(value):
              if pd.notna(value):
                  try:
                      return ast.literal_eval(value).keys()
                  except (SyntaxError, ValueError):
                      return []
              return []

          print("🔍 Extracting all keys...")

          for column_name in column_names:
              if column_name in df.columns:
                  all_keys.update(df[column_name].apply(extract_keys).explode().dropna().unique())

          print(f"✅ Extracted {len(all_keys)} unique keys.")

          # Step 2: Create new columns and fill values
          if TQDM_AVAILABLE:
              progress_bar = tqdm(all_keys, desc="🚀 Expanding values", ncols=80)
          else:
              progress_bar = all_keys

          for idx, key in enumerate(progress_bar):
              df[key] = np.nan
              for column_name in column_names:
                  if column_name in df.columns:
                      df[key] = df[key].combine_first(
                          df[column_name].apply(lambda x: Preprocessing.expand_dict_column(x, key))
                      )

              # Print manual progress if tqdm is not available
              if not TQDM_AVAILABLE and idx % max(1, len(all_keys)//10) == 0:
                  print(f"📈 Progress: {idx}/{len(all_keys)} keys expanded...")

          # Step 3: Drop original dictionary columns
          df = df.drop(columns=column_names, axis=1)

          # Step 4: Save the new file
          output_file_path = output_file_path or file_path.replace(".csv", "_expanded.csv")
          df.to_csv(output_file_path, index=False)

          return f"✅ Columns expanded and file saved successfully at: {output_file_path}"

      except Exception as e:
          return f"❌ An error occurred: {e}"

  @staticmethod
  def expand_dict_column( value, column_name):
      if pd.notna(value):
          try:
              dictionary = ast.literal_eval(value)
              return dictionary.get(column_name, np.nan)
          except (SyntaxError, ValueError):
              return np.nan
      return np.nan


a = Preprocessing()
#a.data_encoding('/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv',
#                column_number=19)

#a.clean_and_save(file_path='/content/eve_del_expanded.csv',
 #                output_path='/content/eve_tag.csv', threshold=80)
#a.expand_dict_columns( file_path='/content/eve_del.csv',
 #                    column_names=['tcp' , 'flow'])
#a.drop_constant_columns(file_path='/content/eve_tag.csv')

c= a.show_column_types( file_path='/content/eve.csv')
a.data_encoding(file_path='/content/eve.csv',
               columns_names=c)

  0%|          | 65/286150 [00:00<13:15, 359.59it/s]

  0%|          | 295/286150 [00:00<12:11, 390.62it/s]

  0%|          | 378/286150 [00:01<11:58, 397.75it/s]

  0%|          | 555/286150 [00:01<11:09, 426.72it/s]

  0%|          | 643/286150 [00:01<10:58, 433.36it/s]

...

  1%|▏         | 3839/286150 [00:27<27:29, 171.16it/s]

  1%|▏         | 3859/286150 [00:27<1:18:44, 59.74it/s]

  1%|▏         | 3917/286150 [00:28<42:18, 111.16it/s]

  1%|▏         | 3954/286150 [00:28<34:23, 136.76it/s]

  1%|▏         | 3968/286150 [00:29<1:12:22, 64.99it/s]

In [87]:
a.columns_drop( file_path='/content/eve_tag_no_constant.csv',
               columns_names=['start', 'end'], output_path='/content/eve.csv')

'Saved cleaned data to /content/eve.csv'

In [81]:
c

['proto', 'event_type', 'end', 'alerted', 'start', 'reason', 'state']

In [ ]:
class NetworkPreprocessing:
  def __init__(self):
    pass


  def extract_tcp_option(self, option, key):
    for opt in option:
      if opt[0] == key:
        return opt[1]
    return None


  def delete_data(self, file_path):
    df = pd.read_csv(file_path)
    df = df.loc[:, df.nunique() > 1]
    df.to_csv(file_path, index=False)
    return f"Columns containing only one value are deleted..."

  def convert_pcap_csv(self, pcap_file, csv_file):
    try:
      print("Countig packages...")
      with PcapReader(pcap_file) as count_reader:
        total_packets = sum(1 for _ in count_reader)
        print(f"Total number of packages: {total_packets}")

      print("Start converting to csv file...")
      with PcapReader(pcap_file) as packets:
        with open(csv_file, mode='w', newline="") as file:
          writer = csv.writer(file)
          writer.writerow([
              "Packet Number", "Ethernet DST", "Ethernet SRC", "Ethernet Type",
              "IP Version", "IP IHL", "IP TOS", "IP Length", "IP ID", "IP Flags",
              "IP Fragmentation", "IP TTL", "IP Proto", "IP Checksum",
              "IP Src", "IP Dst", "TCP Src Port", "TCP Dst Port", "TCP Seq",
              "TCP Ack", "TCP Data Offset", "TCP Reserved", "TCP Flags",
              "TCP Window", "TCP Checksum", "TCP Urgent Pointer", "TCP Options MSS",
              "TCP Options WScale", "TCP Options NOP","Hexdump"
          ])

          for i, packet in enumerate(tqdm(packets,total=total_packets, desc="Packet processing")):
            if packet.haslayer('Ethernet'):
              eth_dst = packet['Ethernet'].dst
              eth_src = packet['Ethernet'].src
              eth_type = packet['Ethernet'].type
            else:
              eth_dst = eth_src = eth_type = None

            if packet.haslayer('IP'):
              ip_version = packet['IP'].version
              ip_ihl = packet['IP'].ihl
              ip_tos = packet['IP'].tos
              ip_len = packet['IP'].len
              ip_id = packet['IP'].id
              ip_flags = packet['IP'].flags
              ip_frag = packet['IP'].frag
              ip_ttl = packet['IP'].ttl
              ip_proto = packet['IP'].proto
              ip_chksum = packet['IP'].chksum
              ip_src = packet['IP'].src
              ip_dst = packet['IP'].dst
            else:
              ip_version = ip_ihl = ip_tos = ip_len = ip_id = ip_flags = ip_frag = ip_ttl = ip_proto = ip_chksum = ip_src = ip_dst = None

            if packet.haslayer('TCP'):
              tcp_sport = packet['TCP'].sport
              tcp_dport = packet['TCP'].dport
              tcp_seq = packet['TCP'].seq
              tcp_ack = packet['TCP'].ack
              tcp_dataofs = packet['TCP'].dataofs
              tcp_reserved = packet['TCP'].reserved
              tcp_flags = packet['TCP'].flags
              tcp_window = packet['TCP'].window
              tcp_chksum = packet['TCP'].chksum
              tcp_urgptr = packet['TCP'].urgptr
              tcp_options = packet['TCP'].options

              tcp_mss = self.extract_tcp_option(tcp_options, 'MSS')
              tcp_wscale = self.extract_tcp_option(tcp_options, 'WScale')
              tcp_nop_count = self.extract_tcp_option(tcp_options, 'NOP')

            else:
              tcp_sport = tcp_dport = tcp_seq = tcp_ack = tcp_dataofs = tcp_reserved = tcp_flags = tcp_window = tcp_chksum = tcp_urgptr = tcp_mss = tcp_wscale = tcp_nop_count = None

            raw_data = packet.original.hex()
            writer.writerow([
                i + 1, eth_dst, eth_src, eth_type,
                  ip_version, ip_ihl, ip_tos, ip_len, ip_id, ip_flags,
                  ip_frag, ip_ttl, ip_proto, ip_chksum,
                  ip_src, ip_dst, tcp_sport, tcp_dport, tcp_seq,
                  tcp_ack, tcp_dataofs, tcp_reserved, tcp_flags,
                  tcp_window, tcp_chksum, tcp_urgptr, tcp_mss, tcp_wscale, tcp_nop_count,
                  raw_data
            ])

      return f"The file has been converted to csv successfully."
    except Exception as e:
      return f"An error occurred: {e}"

a = NetworkPreprocessing()
#a.convert_pcap_csv(pcap_file='/content/NGIDS/NGIDS-DS-v1/NGIDS.pcap',
#                   csv_file='/content/drive/MyDrive/hybrid_IDS/network.csv')
#a.delete_data(file_path='/content/drive/MyDrive/hybrid_IDS/network.csv')

In [ ]:
class PcapFileProcessing:
  def __init__(self) -> None:
    pass



  def delete_data(self, file_path):
    df = pd.read_csv(file_path)
    df = df.loc[:, df.nunique() > 1]
    df.to_csv(file_path, index=False)
    return f"Columns containing only one value are deleted..."


  def date_time_columns(self, file_path, csv_file):
    try:
        df = pd.read_csv(file_path, dtype=str)
        df.columns = df.columns.str.strip()
        if 'timestamp' not in df.columns:
            return "Error: The 'timestamp' column is missing from the file."

        df['timestamp'] = df['timestamp'].astype(str).str.strip()

        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', dayfirst=True)

        if df['timestamp'].isnull().all():
            return "All values in 'timestamp' are invalid and could not be converted."

        # Extract date and time
        df['date'] = df['timestamp'].dt.strftime('%d/%m/%Y')
        df['time'] = df['timestamp'].dt.strftime('%H:%M:%S')

        # Save the result
        df.to_csv(csv_file, index=False)

        return "Successfully converted 'timestamp' to 'date' and 'time'."

    except Exception as e:
        return f"An error occurred during conversion: {e}"





  def convert_log_csv(self, log_file, csv_file):
    try:
      print("Start converting to csv file...")
      with open(log_file, "r") as logfile:
        log_data = logfile.read()
      pattern = re.compile(
          r'(?P<timestamp>\d{2}/\d{2}/\d{4}-\d{2}:\d{2}:\d{2}\.\d+)\s+\[\*\*\]\s+\[(?P<sid>[^]]+)\]\s+(?P<signature>.*?)\s+\[\*\*\]\s+\[Classification:\s+(?P<classification>.*?)\]\s+\[Priority:\s+(?P<priority>\d+)\]\s+\{(?P<protocol>\w+)\}\s+(?P<src_ip>\d+\.\d+\.\d+\.\d+):(?P<src_port>\d+)\s+->\s+(?P<dst_ip>\d+\.\d+\.\d+\.\d+):(?P<dst_port>\d+)'
          )
      with open(csv_file, "w", newline="") as csvfile:
        fieldnames = ['timestamp', 'sid', 'signature', 'classification',
                      'priority', 'protocol','src_ip', 'src_port', 'dst_ip', 'dst_port']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for match in pattern.finditer(log_data):
          writer.writerow(match.groupdict())

      return f"The file has been converted to csv successfully."

    except Exception as e:
      return f"An error occurred: {e}"


  def convert_json_csv(self, json_file, csv_file):
    try:
      print("Start converting JSON to CSV...")
      with open(json_file, 'r') as f:
        data = [json.loads(line) for line in f if line.strip()]

      fieldnames = set()
      for row in data:
          fieldnames.update(row.keys())

      fieldnames = list(fieldnames)

      with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(fieldnames)

        for row in data:
          writer.writerow(row.get(field, '') for field in fieldnames)

      return f"The file has been converted to csv successfully."

    except Exception as e:
      return f"An error occurred: {e}"



a = PcapFileProcessing()
#a.convert_log_csv( log_file="/content/drive/MyDrive/hybrid_IDS/fast.log",
#                  csv_file="/content/drive/MyDrive/hybrid_IDS/alerts.csv")

#a.convert_json_csv(json_file='/content/drive/MyDrive/hybrid_IDS/eve.json',
#                   csv_file='/content/drive/MyDrive/hybrid_IDS/eve.csv')

#a.delete_data( file_path='/content/drive/MyDrive/hybrid_IDS/eve.csv')
a.date_time_columns(file_path='/content/drive/MyDrive/hybrid_IDS/eve.csv',
                    csv_file='/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv')